# To do today:

* ~Read paper sent by Miles (https://www-nature-com.ezproxy.is.ed.ac.uk/articles/s41576-021-00434-9)~
* Work on PyMol alignment script
  * For aligning to original crystal pose
    * Receptor is not needed
    * Generate 20 "conformers" using RDKit (https://iwatobipen.wordpress.com/2021/01/31/generate-conformers-script-with-rdkit-rdkit-chemoinformatics/), align all 20 then choose pose with lowest RMSD to crystal pose.
* Essential Reading
* Read through SCORCH code to understand it

# Notes for today:

* Workflow for PyMol alignmnet:
  * Convert from PDB to mol with RDKit
  * Generate 20 conformers for every active ligand (And 20 for each decoy).
  * Align all 20 to the orginal crystal pose of that ligand
  * Conformers are in form of SDF file
  * Calculate RMSD for each conformer, then chose the conformer with the best pose.
  * Save the one with best pose to PDB file
  * Save the best pose to an output directory "Aligned_PDB_Files"
 * Had to disable sanitize in rdkit "Chem.rdmolfiles.MolFromPDBFile(pdb_file_path, sanitize=False)" due to 91 pdb files not being converted otherwise.

# Tasks completed today:

* Wrote a script to convert all smiles_pdb files to ".mol" files, as rdk_confgen needs .mol files as input. Saved below as "pdb_to_mol.py"
* Made a "pdb_confgen.py" script (code below) to create 20 conformers for every molecule.
  * Having issues with RDKit in confgen, will need to edit confen.py file

## pdb_to_mol.py

In [1]:
from rdkit import Chem
import os
from joblib import Parallel, delayed

output_dir = "/home/s2451611/MScProject/pdb_to_mols_dir"
os.makedirs(output_dir, exist_ok=True)

def pdb_to_mol(pdb_file_path):
    molecule = Chem.rdmolfiles.MolFromPDBFile(pdb_file_path, sanitize=False)
    
    # These lines can be activated if Chem.rdmolfiles.MolFromPDBFile() is returning a None type to find which pdb files are causing an issue
    # if molecule is None:
    #     with open("broken_pdb.txt", "a")as f:
    #         f.write(f"Could not convert {pdb_file_path} to a molecule.\n")
    #     continue
    mol_file = os.path.basename(pdb_file_path).rstrip(".pdb")
    Chem.rdmolfiles.MolToMolFile(molecule, f"{output_dir}/{mol_file}.mol")

def main():
    input_dirs = ["/home/s2451611/MScProject/Acitve_Smiles_pdbs", "/home/s2451611/MScProject/Decoy_Smiles_pdbs"]

    for input_dir in input_dirs:
        pdb_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir)]
        Parallel(n_jobs=-1)(delayed(pdb_to_mol)(pdb_file) for pdb_file in pdb_files)

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'rdkit'

In [ ]:
## pdb_confgen.py

In [ ]:
from rdkit import Chem
import os
from joblib import Parallel, delayed

output_dir = "/home/s2451611/MScProject/conformer_dir/"
input_dir = "/home/s2451611/MScProject/pdb_to_mols_dir/"

def confgen(file_arg):
    output_file_name = os.path.basename(file_arg).rstrip(".mol") + ".sdf"
    output_file = output_dir + output_file_name
    os.system(f"confgen -i {file_arg} -o {output_file}")

def main():
    os.makedirs(output_dir, exist_ok=True)
    file_names = os.listdir(input_dir)
    Parallel(n_jobs=-1)(delayed(confgen)(os.path.join(input_dir, file_name)) for file_name in file_names)

if __name__ == "__main__":
    main()


# To do tomorrow:

* Work on confgen to get working correctly (https://www.rdkit.org/UGM/2012/Ebejer_20110926_RDKit_1stUGM.pdf)
* Work on PyMol alignment
* Essential reading
* Read SCORCH code
* Edit "pdb_confgen.py" to include 20 conformers not 50